In [2]:
import pandas as pd
import numpy as np

In [3]:
import pymysql
import pymssql

/home/adarsh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [4]:
import sys

In [5]:
sys.path.insert(0,'/home/shared/utils')

In [6]:
from db_utils import *


In [7]:
iloans_conn = get_iloans_conn()
predicon_conn = get_predicon_db_conn()
bank_app_conn = get_bank_app_conn()

In [161]:
query_bank_app_reviewed = '''
select loan_id As LoanId, 
final_decision,
reasons_for_decision,
entered_date,
path                             
from loan 
where campaign like '%Production%'
and STR_TO_DATE(entered_date ,'%m/%d/%Y') >= STR_TO_DATE('06/09/2020','%m/%d/%Y')
and STR_TO_DATE(entered_date ,'%m/%d/%Y') < STR_TO_DATE('06/21/2020','%m/%d/%Y')
'''

In [162]:
df_bank_app_reviewed = pd.read_sql_query(query_bank_app_reviewed,con=bank_app_conn)

In [164]:
rename_columns = {'final_decision':'manual_decision',
                  'reasons_for_decision':'manual_decision_reason'}
df_bank_app_reviewed = df_bank_app_reviewed.rename(columns = rename_columns)

In [8]:
df_scores_latest = pd.read_sql_query('''SELECT * FROM 
                                     FreedomScores 
                                     WHERE Version='v1.3.0' 
                                     GROUP BY LoanId''',con= predicon_conn)

In [157]:
df_scores_latest['LoanId'] = df_scores_latest['LoanId'].astype(str) 

In [158]:
df_scores_latest = df_scores_latest.drop(columns=['EsigTimeSignedDiff_In_SEC'])

In [166]:
df_merge_bank_review_score = pd.merge(df_bank_app_reviewed,df_scores_latest,how='left',on='LoanId')

In [170]:
query_funded = '''
SELECT LoanId 
,LoanStatus
FROM view_FCL_LoanAll
WHERE LeadTimeAdded >= '2020-06-08'
'''

In [171]:
df_funded_loans = pd.read_sql_query(query_funded,con=iloans_conn)

In [173]:
df_funded_loans['LoanId'] = df_funded_loans['LoanId'].astype(int).astype(str)

In [174]:
df_merge_bank_review_score_funded = pd.merge(
    df_merge_bank_review_score,
    df_funded_loans,
    on='LoanId',how='left')

In [177]:
cols = ['LoanId','manual_decision','entered_date','path','Score','LoanStatus']
df_score_analysis = df_merge_bank_review_score_funded[cols]

In [179]:
is_score_positive = df_score_analysis['Score'] <=0.51

In [ ]:
df_score_analysis['model_decision'] = np.nan

is_score_positive = (df_score_analysis['Score'] <=0.51)
is_score_neutral = (df_score_analysis['Score'] > 0.51)
no_score = df_score_analysis['Score'].isnull()

df_score_analysis.loc[is_score_positive,'model_decision'] = 'Positive'
df_score_analysis.loc[is_score_neutral,'model_decision'] = 'Neutral'
df_score_analysis.loc[no_score,'model_decision'] = 'No score'

In [ ]:
df_score_analysis.loc[:,'entered_date'] = pd.to_datetime(df_score_analysis['entered_date'],format = "%m/%d/%Y")

In [ ]:
df_score_analysis.to_csv('score_analysis.csv')